In [1]:
#import packages 

from glob import glob
import pandas as pd
import os
import shutil
import math

folder_path="./dataset/subset/"
folders = glob(folder_path+'complete/*')

train_split=0.7
no_of_images=0

In [14]:
#split data to training and testing

def createFolder(path):
	if os.path.exists(path):
		shutil.rmtree(path)
	os.makedirs(path)

createFolder(folder_path+"train")
createFolder(folder_path+"test")

for folder in folders:
    images = glob(folder+'/*.jpg')
    name = folder.split('/')[-1]
    createFolder(folder_path+"train/"+name)
    createFolder(folder_path+"test/"+name)
    train_images=math.floor(train_split*len(images))
    for i,image in enumerate(images):
        if(i<=train_images-1):
            shutil.copy(image,folder_path+"train/"+name)
        else:
            shutil.copy(image,folder_path+"test/"+name)
        no_of_images+=1
    print(name)


complete\BabyHat
complete\BabyPants
complete\BabyShirt
complete\womencasualshoes
complete\womenlongsleevetop


In [2]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers.core import Dense, Dropout


# Initialising the CNN
classifier = Sequential()
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.2))
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.2))
# Step 3 - Flattening
classifier.add(Flatten())
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 5, activation = 'softmax'))
# Compiling the CNN
classifier.compile(optimizer = 'adam',
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])
classifier.summary()


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
__________

In [ ]:

# Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   horizontal_flip=True,
                                   width_shift_range= 0.3,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(folder_path+"train/",
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(folder_path+"test/",
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

classifier.fit_generator(training_set,steps_per_epoch = 12500/32,
                         epochs = 15,
                         validation_data = test_set,
                         validation_steps = 30)

Found 8751 images belonging to 5 classes.
Found 3753 images belonging to 5 classes.
Epoch 1/15
391/390 [==============================] - 132s 337ms/step - loss: 1.4814 - acc: 0.3695 - val_loss: 1.3684 - val_acc: 0.4688
Epoch 2/15
391/390 [==============================] - 48s 122ms/step - loss: 1.3247 - acc: 0.4657 - val_loss: 1.1535 - val_acc: 0.5375
Epoch 3/15
391/390 [==============================] - 41s 106ms/step - loss: 1.1840 - acc: 0.5360 - val_loss: 0.9817 - val_acc: 0.6573
Epoch 4/15
391/390 [==============================] - 40s 103ms/step - loss: 1.0896 - acc: 0.5845 - val_loss: 0.9705 - val_acc: 0.6467
Epoch 5/15
391/390 [==============================] - 37s 95ms/step - loss: 1.0091 - acc: 0.6143 - val_loss: 0.8235 - val_acc: 0.7000
Epoch 6/15
381/390 [============================>.] - ETA: 0s - loss: 0.9938 - acc: 0.6222